# Algoritmo 3 - Associação de Regras

Vamos usar o Apriori para encontrar associações entre os desportos praticados pelos atletas olímpicos.

## Introdução
Este notebook aplica mineração de regras de associação sobre resultados olímpicos para identificar combinações relevantes entre modalidades e medalhas. Começamos pela consolidação dos ficheiros Summer e Winter, enriquecemos os registos com dados demográficos e, por fim, extraímos padrões frequentes com o algoritmo Apriori.

In [10]:
print("Hello World!")

# Instala/atualiza dependências usadas no notebook
%pip install --upgrade pip
%pip install numpy
%pip install matplotlib
%pip install pandas
%pip install mlxtend


Hello World!
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to u

## Preparação do ambiente
As linhas seguintes asseguram que todas as bibliotecas necessárias (pandas, numpy, matplotlib e mlxtend) estão instaladas e atualizadas. Executa-as apenas quando configurares o notebook pela primeira vez ou sempre que mudares de ambiente.

# Importar Ficheiro CSV
Carregamos os ficheiros `winter.csv` e `summer.csv`, adicionamos a coluna `Season` para distinguir as observações e fazemos o `concat` para ter um único dataset. Em seguida, renomeamos `Country` para `Country_Code` para alinhar com o dicionário e juntamos informação demográfica como população e PIB per capita.

In [11]:
import pandas as pd

# Carregar Jogos de Inverno e etiquetar temporada
winterData = pd.read_csv("Data/Olympics/winter.csv")
print(winterData.head())

winterData["Season"] = "Winter"
print(winterData.head())


# Carregar Jogos de Verão e etiquetar temporada
summerData = pd.read_csv("Data/Olympics/summer.csv")
print(summerData.head())

summerData["Season"] = "Summer"
print(summerData.head())


# Unificar datasets sazonais
concatData = pd.concat([winterData, summerData], ignore_index=True)

print("---------------------------------------------")

# Harmonizar código do país para corresponder ao dicionário
concatData.rename(columns={"Country": "Country_Code"}, inplace=True)
print(concatData.tail())

dictionary = pd.read_csv("Data/Olympics/dictionary.csv")

# Enriquecer com população e PIB per capita
olympics = concatData.merge(
    dictionary[["Code","Country", "Population", "GDP per Capita"]],
    left_on="Country_Code",   # HUN, AUT, GRE, ...
    right_on="Code",     # HUN, AUT, GRE, ...
    how="left"
)



   Year      City     Sport Discipline               Athlete Country Gender  \
0  1924  Chamonix  Biathlon   Biathlon           BERTHET, G.     FRA    Men   
1  1924  Chamonix  Biathlon   Biathlon        MANDRILLON, C.     FRA    Men   
2  1924  Chamonix  Biathlon   Biathlon   MANDRILLON, Maurice     FRA    Men   
3  1924  Chamonix  Biathlon   Biathlon       VANDELLE, André     FRA    Men   
4  1924  Chamonix  Biathlon   Biathlon  AUFDENBLATTEN, Adolf     SUI    Men   

             Event   Medal  
0  Military Patrol  Bronze  
1  Military Patrol  Bronze  
2  Military Patrol  Bronze  
3  Military Patrol  Bronze  
4  Military Patrol    Gold  
   Year      City     Sport Discipline               Athlete Country Gender  \
0  1924  Chamonix  Biathlon   Biathlon           BERTHET, G.     FRA    Men   
1  1924  Chamonix  Biathlon   Biathlon        MANDRILLON, C.     FRA    Men   
2  1924  Chamonix  Biathlon   Biathlon   MANDRILLON, Maurice     FRA    Men   
3  1924  Chamonix  Biathlon   Biath

## Construir transações
Criamos a variável `SportMedal` para combinar desporto e medalha e agregamos por país/ano/temporada para obter o conjunto de itens por participação olímpica. Apenas mantemos transações com pelo menos um item antes de aplicar o `TransactionEncoder`.

In [12]:
from mlxtend.preprocessing import TransactionEncoder


# Criar item combinando desporto e medalha
olympics["SportMedal"] = olympics["Sport"] + "_" + olympics["Medal"]

# Agregar itens por país/ano/temporada
transactions_df = (
    olympics
    .groupby(["Country", "Year", "Season"])["SportMedal"]
    .unique()
    .reset_index()
)


# Descartar participações vazias e gerar lista de transações
transactions_df["num_items"] = transactions_df["SportMedal"].apply(len)
transactions_df = transactions_df[transactions_df["num_items"] >= 1].reset_index(drop=True)
transactions = transactions_df["SportMedal"].tolist()


# Transformar para matriz one-hot
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
basket = pd.DataFrame(te_ary, columns=te.columns_)


## Descobrir regras de associação
Aplicamos o `apriori` para gerar `frequent_itemsets` com suporte mínimo de 2% e, em seguida, calculamos regras com confiança mínima de 0.5. Filtramos as regras simples (1 antecedente → 1 consequente) com suporte relevante e `lift` > 1.5 para destacar relações realmente interessantes entre desporto e medalha.

In [13]:
from mlxtend.frequent_patterns import apriori, association_rules


# Extrair conjuntos frequentes e regras com thresholds base
frequent_itemsets = apriori(basket, min_support=0.02, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

rules["antecedent_len"] = rules["antecedents"].apply(len)
rules["consequent_len"] = rules["consequents"].apply(len)

# Destacar regras simples com suporte e lift relevantes
simple_rules = rules[
    (rules["antecedent_len"] == 1) &    # antecedente só com 1 item
    (rules["consequent_len"] == 1) &    # consequente com 1 item (se quiseres simples)
    (rules["support"] >= 0.02) &        # suporte mínimo
    (rules["lift"] > 1.5)               # lift para garantir associação interessante
].sort_values("lift", ascending=False)



rules["antecedent_len"] = rules["antecedents"].apply(len)
rules["consequent_len"] = rules["consequents"].apply(len)

# Ordenar todas as regras por lift para exploração adicional
rules_sorted = rules.sort_values("lift", ascending=False)
rules_sorted.head(20)
print(simple_rules.head(20))


                antecedents             consequents  antecedent support  \
108     (Equestrian_Silver)       (Equestrian_Gold)            0.066809   
109       (Equestrian_Gold)     (Equestrian_Silver)            0.051173   
100    (Canoe / Kayak_Gold)  (Canoe / Kayak_Bronze)            0.049751   
119     (Gymnastics_Silver)     (Gymnastics_Bronze)            0.068941   
118     (Gymnastics_Bronze)     (Gymnastics_Silver)            0.078891   
111        (Fencing_Bronze)          (Fencing_Gold)            0.074627   
110          (Fencing_Gold)        (Fencing_Bronze)            0.063966   
114        (Fencing_Silver)          (Fencing_Gold)            0.076048   
115          (Fencing_Gold)        (Fencing_Silver)            0.063966   
117       (Gymnastics_Gold)     (Gymnastics_Bronze)            0.066809   
116     (Gymnastics_Bronze)       (Gymnastics_Gold)            0.078891   
124     (Gymnastics_Silver)       (Gymnastics_Gold)            0.068941   
123       (Gymnastics_Gol

## Interpretar resultados
O dataframe `simple_rules` contém as associações mais fortes (lift > 1.5) entre desporto e medalha, enquanto `rules_sorted` lista o top 20 ordenado por `lift`. Analisa estas tabelas para identificar padrões consistentes (por exemplo, modalidades que costumam resultar em determinadas medalhas) e decide quais regras merecem destaque no relatório final.

## Conclusão
O pipeline consolidado permitiu descrever o panorama olímpico ao nível de país, edição e temporada, revelando combinações de desporto-medalha com padrões consistentes. As regras com maior lift ajudam a direcionar a análise para modalidades onde a probabilidade de medalha é mais elevada, servindo de base para recomendações estratégicas ou storytelling no relatório final. Ajusta os limiares de suporte/confiança conforme necessário para aprofundar segmentos específicos ou testar hipóteses adicionais.